In [ ]:
import collections

import tensorflow as tf
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
%matplotlib inline

In [ ]:
def load_imgs(dir_path):
    name_list = os.listdir(dir_path)
    
    img_list = []
    
    for name in name_list:
        img_path = "{}/{}".format(dir_path, name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(556,360))
        img_list.append(img)
    return img_list

In [ ]:
img_list = load_imgs("./data")

In [ ]:
plt.imshow(img_list[0])

In [ ]:
def blur_img_list(img_list):
    result_list =[]
    for img in img_list:
        result = cv2.GaussianBlur(img,(15,15), 3)
        result_list.append(result)
    return result_list

In [ ]:
X_train = np.array(blur_img_list(img_list))
Y_train = np.array(img_list)

In [ ]:
plt.imshow(X_train[10])

In [ ]:
plt.imshow(Y_train[10])

In [ ]:
tf.reset_default_graph()

In [ ]:
def conv2d(X, n_input, n_output, filter_size, name):
    
    with tf.variable_scope(name):
        W = tf.get_variable(
                    name='W_1',
                    shape=[filter_size[0],filter_size[1],n_input,n_output],
                    initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01))

        b = tf.get_variable(
                    name='b_1',
                    shape=[n_output],
                    initializer=tf.constant_initializer())

        conv = tf.nn.conv2d(X,
                            W, 
                            strides= [1,1,1,1],
                            padding='SAME'
                           )
        h = tf.nn.relu(tf.nn.bias_add(conv,b))
    
    return h

In [ ]:
def build_model(learning_rate=0.001):
    
    height = 360
    width = 556
    channel = 3
    n_flat = height * width * channel
    X = tf.placeholder(tf.float32, shape=[None, height,width,channel],name='X')
    Y = tf.placeholder(tf.float32, shape=[None, height,width,channel], name='Y')
    
    layer_info_list = [
        {'name':'conv_1',
                    'n_input':3,
                    'n_output':16,
                    'filter_size':(3,3)},
        {'name':'conv_2', 
                    'n_input':16,
                    'n_output':32,
                    'filter_size':(3,3)
                   },
        {'name':'conv_5',
                    'n_input':32,
                    'n_output':3,
                    'filter_size':(3,3)
                   },
            ]
    
    current_input = X
    
    for info in layer_info_list:
        
        current_input = conv2d(X=current_input,
                                   n_input=info['n_input'],
                                   n_output=info['n_output'],
                                   filter_size=info['filter_size'],
                                   name=info['name'],
                                  )
    Y_pred = current_input
    
    Y_pred_flattened = tf.reshape(Y_pred, shape=[-1,n_flat ])
    Y_flattened = tf.reshape(Y, shape=[-1,n_flat])
    
    cost = tf.reduce_mean(tf.reduce_sum(tf.squared_difference(Y_pred, Y),axis=1), axis=0)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    return X, Y, Y_pred, cost, optimizer,
        

In [ ]:
def train(X_train, Y_train, X, Y, Y_pred, cost, optimizer):

    batch_size = 6
    n_epoch = 10

    with tf.Session() as sess:
            
        sess.run(tf.global_variables_initializer())
    
        for epoch_i in range(n_epoch):
            rand_idx = np.random.permutation(range(len(X_train)))
            n_batch = len(rand_idx) // batch_size
            for batch_i in range(n_batch):

                rand_idx = rand_idx[batch_i*batch_size : (batch_i+1)*batch_size]
                batch_x = X_train[rand_idx]
                batch_y = Y_train[rand_idx]
                sess.run(optimizer, feed_dict={X:batch_x, Y: batch_y})

    
            Y_train_pred = Y_pred.eval(feed_dict={X:X_train[0] },session=sess)
            training_cost = sess.run(cost, feed_dict={X:X_train,Y:Y_train})
            plt.imshow(Y_train_pred)
            print(training_cost)

In [ ]:
X, Y, Y_pred, cost, optimizer = build_model()

In [ ]:
train(X_train, Y_train, X, Y, Y_pred, cost, optimizer)